# Chapter 11: Complex Structured Output

**Scenario:** You are building an automated ordering system for **Green Bites**, a healthy salad bowl shop. Customers send messy messages like "I want a kale salad with extra avocado, oh and a ginger kombucha, send it to my office at 123 Main St". You need to turn this into a structured JSON for your kitchen and delivery systems.

In this notebook, you will learn:
- 🕸️ **Nested Structure**: How to get the AI to output complex, multi-level JSON.
- 📋 **Strict Schemas**: Using specific instructions to ensure keys match your database.
- 🧠 **Thinking Blocks**: Forcing the AI to "reason" before outputting data to improve accuracy.

---

## 1. Setup

Standard setup with `litellm`.

In [1]:
# Install tools (if needed)
%pip install -q litellm python-dotenv

/Users/param/learn/learnwithparam/lwp-workshops/practical-prompt-engineering-workshop/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# 1. Hide messy tech logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

# 2. Load secret keys
load_dotenv()

# 3. Pick our model
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')
print(f"Ready to process orders with: {MODEL_NAME}")

Ready to process orders with: openrouter/google/gemini-2.0-flash-001


In [3]:
def get_completion(prompt, system_prompt=None, prefill=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    content = response.choices[0].message.content
    if prefill:
        return prefill + content
    return content

## 2. Nested Structured Data

A simple flat list isn't always enough. For **Green Bites**, we need an `order` object that contains `items`, and a `customer` object that contains `address`.

If you just ask the AI for JSON, it might get the keys wrong. You must define the **Schema**.

In [4]:
customer_message = """
Hi! I'd like to get the Summer Berry Salad with extra strawberries. 
Also add a Green Detox Juice. 
My name is Sarah Connor and I'm at 742 Evergreen Terrace. 
Phone is 555-0199. Keep the change!
"""

prompt = f"""
Extract the order details from the customer message into a nested JSON object.

The JSON should follow this structure:
{
  "order_details": {
    "items": [{"name": "item name", "customization": "modifications if any"}],
    "notes": "any special requests"
  },
  "customer_info": {
    "name": "customer name",
    "delivery_address": "full address",
    "phone": "phone number"
  }
}

Customer Message:
<message>
{customer_message}
</message>

Output ONLY the JSON.
"""

response = get_completion(prompt, prefill="{")
print(response)

SyntaxError: f-string: expressions nested too deeply (3916009834.py, line 30)

## 3. Improving Accuracy with "Thinking Blocks"

When you ask an AI to output complicated data immediately, it might make "hallucination" errors because it's focusing too much on the formatting.

**The Pro Technique:** Ask the model to "think" first in its own block, then output the JSON. This forces it to process the logic before committing to a format.

Let's see the comparison.

In [ ]:
complex_input = """
Order for John Wick: I want 2 Classic Caesars but remove the croutons on one of them. 
And two large Sparkling Waters. Oh, wait, make it 3 Caesars total, the third one is normal. 
Deliver to Continental Hotel Room 101.
"""

prompt = f"""
You are an order processing AI for Green Bites.
Process the following messy order.

STEPS:
1. In a <thinking> block, analyze the message, count the quantities, and identify specific customizations.
2. Output the final data in a <json> block.

Order Message:
{complex_input}
"""

response = get_completion(prompt)
print(response)

**Why use tags?** 
When you have both `<thinking>` and `<json>` tags, your code can easily extract the clean data while the AI gets the "scratchpad" it needs to be accurate.

## 4. Exercise: The Feedback Analyzer

**Goal:** Build a prompt that analyzes customer feedback and outputs a multi-part JSON.

**Requirements:**
- **Thinking Block**: Identify the main complaint/praise.
- **JSON Output**: Containing:
  - `sentiment`: "Positive", "Negative", or "Neutral"
  - `category`: "Food", "Service", or "Delivery"
  - `points`: A list of the specific issues mentioned.

**Feedback:** "The salad was delicious but it took over an hour to arrive and the driver didn't call me when they were outside."

In [ ]:
feedback = "The salad was delicious but it took over an hour to arrive and the driver didn't call me when they were outside."

prompt = f"""
Analyze this feedback.
1. Think about the sentiment and key issues in <thinking> tags.
2. Output JSON in <json> tags with keys: sentiment, category, points.

Feedback: {feedback}
"""

response = get_completion(prompt)
print(response)

if "<json>" in response and "sentiment" in response and "category" in response:
    print("\n✅ Great structure!")
else:
    print("\n❌ Make sure to use both <thinking> and <json> tags.")

## Summary

1.  **Define the Schema**: Show the AI exactly what JSON structure you expect.
2.  **Use Thinking Blocks**: Help the AI "slow down" and process logic before writing data.
3.  **Use XML tags for parsing**: Surround the data with tags (`<json>`, `<thinking>`) to easily extract it in your application.